In [2]:
import logging
import zipfile
from pathlib import Path
from datetime import datetime, date
from abc import ABC, abstractmethod

import numpy as np
import pdfplumber
import pandas as pd

import scrapper as sp


class DataReader:
    def __init__(self) -> None:
        self.data_folder = sp.Scrapper().create_or_get_folder()

    def get_file_reader(self):
        dictionary = Path(self.data_folder)

        files_by_extension = {}
        # iter over all the files in the directory
        for file in dictionary.iterdir():
            if file.is_file():
                file_extension = file.suffix.lower()
                # Checking if the extension is in the dictionary
                if file_extension in files_by_extension:
                    files_by_extension[file_extension].append(file)
                else:
                    files_by_extension[file_extension] = [file]

        return files_by_extension

    def read_zip_file(self):
        pass


class DataPreprocessor:
    def get_mkt_summary(self):
        pass

    def get_indhist(self):
        pass

    def get_omts(self):
        pass

    def get_quote(self):
        pass

    def get_open_interest(self):
        pass



In [14]:
dd = DataReader()
files_ = dd.get_file_reader()

In [15]:
files_

{'.log': [PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/scraper.log')],
 '.pdf': [PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/quote2024-01-23.pdf')],
 '.csv': [PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/omts2024-01-23.csv')],
 '.xls': [PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/indhist2024-01-23.xls'),
  PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/fut_opn_int2024-01-23.xls')],
 '.z': [PosixPath('/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/mkt_summary2024-01-23.Z')]}

In [17]:
import re

In [20]:
for file_ext, files in files_.items():
    for file in files:
        s_file = file.stem
        update_file = re.sub(r'[^A-Za-z_]', '', s_file)
        print(update_file)
        

scraper
quote
omts
indhist
fut_opn_int
mkt_summary


In [49]:
from zipfile import ZipFile

with ZipFile("/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/20240125_new (1).lis.Z", "r") as zp:
    zp.extractall("/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/")



In [52]:

fp = "/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/closing11.lis"

df = pd.read_csv(fp, sep='|', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,25JAN2024,AABS,826,Al-Abbas SugarXD,486.02,486.02,486.01,486.02,300,494.39,NaN,NaN,NaN
1,25JAN2024,ABL,807,Allied Bank Ltd,86.00,86.00,86.00,86.00,500,85.79,NaN,NaN,NaN
2,25JAN2024,ABOT,823,Abbott Lab.,430.00,430.00,425.10,425.10,2000,430.00,NaN,NaN,NaN
3,25JAN2024,ACIETF,837,Alfalah Consumer,9.81,9.81,9.81,9.81,5500,9.84,NaN,NaN,NaN
4,25JAN2024,ACPL,804,Attock Cement,97.49,98.90,96.00,97.03,136160,97.00,NaN,NaN,NaN


In [102]:
import logging, io
from tqdm import tqdm
import pandas as pd
import pdfplumber
from datetime import date

In [107]:
def open_pdf_file(file_path):
    try:
        with open(file_path, 'rb') as file:
            file_content = file.read()
        with pdfplumber.open(io.BytesIO(file_content)) as target_pdf_file:
            return target_pdf_file
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {file_path} was not found.")



def extract_page_numbers(pdf_pages):
    target_table_pattern = [
        "SECTION 1: MACRO VIEW OF THE MARKET",
        "SECTION 5: BOARD MEETINGS",
        "SECTION 12: ALL SHARES INDEX REPORT (SECTOR WISE)",
        "SECTION 13: MAIN BOARD DATA FOR THE LAST 6 MONTHS",
    ]
    try:
        targeted_pages_nums = {}
        pages = pdf_pages.pages[1:]
        for page_num, page in tqdm(enumerate(pages, start=1)):
            text = page.extract_text()
            for pattern in target_table_pattern:
                if pattern in text:
                    if pattern not in targeted_pages_nums:
                        targeted_pages_nums[pattern] = []
                    targeted_pages_nums[pattern].append(page_num)

        return targeted_pages_nums
    except Exception as e:
        logging.error(f"Error Message: {e}")

def extract_macro_view(targeted_pages_nums: list, target_pdf_file) -> pd.DataFrame:
    """
    Extracts a macro view of the targeted pages from a PDF file and returns the result as a pandas DataFrame.
    """
    try:
        tables = target_pdf_file.pages[targeted_pages_nums[0] - 1].extract_table()
        loop_ending_pattern = "PUBLICLY ISSUED DEBT SECURITIES"

        df = []

        for table in tables:
            table = [cell for cell in table if cell is not None]
            if any(loop_ending_pattern in cell for cell in table):
                break
            df.append(table)

        df = pd.DataFrame(df)
        df["Date"] = str(date.today())
        df.dropna(inplace=True)
        columns = ["Date","Description", "Main Board", "Description", "GEM Board"]
        df.columns = columns
        df.replace(",", "", regex=True, inplace=True)

        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")

In [95]:
pdf_file = open_pdf_file("/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/quote2024-01-26.pdf")


In [96]:
extract_page_numbers(pdf_file)

42it [00:47,  1.12s/it]


{'SECTION 1: MACRO VIEW OF THE MARKET': [1],
 'SECTION 5: BOARD MEETINGS': [30],
 'SECTION 12: ALL SHARES INDEX REPORT (SECTOR WISE)': [36, 37],
 'SECTION 13: MAIN BOARD DATA FOR THE LAST 6 MONTHS': [37]}

In [108]:

extract_macro_view([2],open_pdf_file("/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/quote2024-01-26.pdf"))

,Date,Description,Main Board,Description,GEM Board
2,No. of Listed Companies,524,No. of Listed Companies,3,2024-01-28
3,Listed Paid up Capital,1669727 M,Listed Paid up Capital,1832 M,2024-01-28
4,Market Capitalisation,9359225920149,Market Capitalisation,2043409524,2024-01-28
5,Traded Volume,412823322,Traded Volume,2500,2024-01-28
6,Traded Value,16185873014,Traded Value,25160,2024-01-28
7,Advancers,99,Advancers,0,2024-01-28
8,Decliners,221,Decliners,1,2024-01-28
9,Neutral,22,Neutral,2,2024-01-28
10,Total,342,Total,3,2024-01-28


In [35]:
def extract_event_table(targeted_pages_nums: list):
    """Extracts a table from the PDF based on the page numbers provided."""
    try:
        if len(targeted_pages_nums) == 1:
            target_table = pdf_file.pages[
                targeted_pages_nums[0] - 1
            ].extract_table()  # -1 as page numbers start from 1 not 0
            tabledf = pd.DataFrame(target_table)
            tabledf.dropna(inplace=True)
            tabledf.columns = tabledf.iloc[0]
            tabledf.drop(tabledf.index[0], inplace=True)
            df = tabledf.iloc[:, 1:]
        else:
            df = pd.DataFrame()
            print(len(targeted_pages_nums))
            for page in range(targeted_pages_nums[0] - 1, targeted_pages_nums[1]):
                extracted_table_page = pdf_file.pages[page]
                extracted_table = extracted_table_page.extract_table()
                extracted_table = pd.DataFrame(extracted_table)
                extracted_table.dropna(inplace=True)
                
                extracted_table.columns = extracted_table.iloc[0]
                extracted_table.drop(extracted_table.index[0], inplace=True)
                table = extracted_table.iloc[:, 1:]
                df = pd.concat([df, table], axis=0)
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")
    return df

In [36]:
extract_event_table([30])

1,Company,Date,Time,Place,Subject
2,Habib Sugar Mills Limited,24-Jan-2024,13:00,Karachi,1st Quarterly Accounts for the Period ended De...
3,Lucky Core Industries Limited,24-Jan-2024,14:30,Karachi,Half Yearly Accounts for the Period ended Dece...
4,Archroma Pakistan Limited,24-Jan-2024,11:15,Karachi,1st Quarterly Accounts for the Period ended De...
5,Honda Atlas Cars (Pakistan) Limited,24-Jan-2024,11:00,Karachi,3rd Quarterly Accounts for the Period ended De...
6,Gharibwal Cement Limited,24-Jan-2024,14:00,Lahore,To consider the matter other than Financial Re...
7,UBL-Funds,25-Jan-2024,11:30,Karachi,Half Yearly Accounts for the Period ended Dece...
8,Saudi Pak Leasing Company Limited,25-Jan-2024,11:00,Karachi,To consider the matter other than Financial Re...
9,Abdullah Shah Ghazi Sugar Mills Limited,25-Jan-2024,10:00,Lahore,Annual Accounts for the Period ended September...
10,Saudi Pak Leasing Company Limited,25-Jan-2024,11:00,Karachi,To consider the matter other than Financial Re...
11,Fauji Fertilizer Bin Qasim Limited,25-Jan-2024,11:00,Islamabad,Annual Accounts for the Period ended December ...


In [37]:
def six_month_market_summary(targeted_pages_nums):
    """
    Extracts the six-month market summary from a given PDF file.
    Args:
        targeted_pages_nums (List[int]): A list of page numbers to extract tables from.
    Returns:
        pandas.DataFrame: A DataFrame containing the extracted market summary data.
    """
    try:
        tables = pdf_file.pages[targeted_pages_nums[0] - 1].extract_table()

        target_table = []
        start_appending = False
        for table in tables:
            if (
                table[0] is not None
                and "SECTION 13: MAIN BOARD DATA FOR THE LAST 6 MONTHS" in table[0]
            ):
                start_appending = True
            if start_appending:
                clean_table = [
                    cell for cell in table if cell is not None
                ]  # remove empty cells (None values)
                clean_table = [
                    re.sub(r"\s+", " ", clean_table) for clean_table in clean_table
                ]  # remove leading and trailing spaces
                target_table.append(clean_table)

                if "SECTION 14: DEFAULTER SEGMENT" in clean_table:
                    break

        df = pd.DataFrame(
            target_table,
            columns=[
                "Month at the Close",
                "Listed Capital(million)",
                "Market Capitalization(million)",
                "Turnover In Ready MRKT",
                "Turnover In Future MRKT",
                "KSE 100 Index",
                "KSE All Share",
            ],
        )
        df.dropna(inplace=True)
        df.replace(",", "", regex=True, inplace=True)
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")

In [39]:
import re

In [40]:
six_month_market_summary([37])

,Month at the Close,Listed Capital(million),Market Capitalization(million),Turnover In Ready MRKT,Turnover In Future MRKT,KSE 100 Index,KSE All Share
3,JULY,1628368.120,7231764.255,7984.831,2501.287,48034.600,32027.290
4,AUGUST,1629437.234,6715880.490,6814.516,2511.124,45002.420,30041.970
5,SEPTEMBER,1637300.480,6885570.255,3447.734,1621.924,46232.590,31158.710
6,OCTOBER,1653218.018,7551772.884,8404.818,2531.777,51920.270,34558.220
7,NOVEMBER,1656800.393,8729523.266,14056.434,3686.656,60531.270,40324.280
8,DECEMBER,1665476.972,9062902.809,21165.662,5347.305,62451.040,41916.270


In [45]:
def get_omts(file_path):
    """
           This method receives a CSV file path and splits the data into two separate pd.DataFrames.

           Parameters:
               file_path (str): CSV file path to read.

           Returns:
               df1: The first processed DataFrame containing off-market BROKER TO BROKER TRADES.
               df2: The second processed DataFrame containing CROSS TRANSACTIONS BETWEEN CLIENT TO CLIENT & FINANCIAL INSTITUTIONS
           """
    try:
        df = pd.read_csv(file_path, skip_blank_lines=True, skiprows=4)
        # Find the index of the first row where any column is NaN
        separator_index = df[df.isnull().any(axis=1)].index[0]
        # Slice the DataFrame into two separate DataFrames using this index
        df1 = df.loc[: separator_index - 1].copy()
        df2 = df.loc[separator_index + 2:].copy()  # Skip the first row directly
        df1.reset_index(
            drop=True, inplace=True
        )  # reset the index to 0,1,2… after slicing
        df2.reset_index(drop=True, inplace=True)
        df1.columns = df1.columns.str.strip()
        df2.columns = df2.columns.str.strip()
        # split the 'MEMBER CODE' column literal string search
        df1[["0", "BUYER", "2", "SELLER"]] = df1["MEMBER CODE"].str.split(
            " ", expand=True
        )
        # Drop the unnecessary columns
        df1.drop(columns=["MEMBER CODE", "0", "2"], inplace=True)
        df1 = df1[
            [
                "Date",
                "SETTLEMENT DATE",
                "BUYER",
                "SELLER",
                "SYMBOL CODE",
                "COMPANY",
                "TURNOVER",
                "RATE",
                "VALUES",
            ]
        ]  # reorder the columns
        return df1, df2
    except Exception as e:
        print(f"Error Message: {e}")

In [46]:
df1, df2 = get_omts("/Volumes/PSXDatabase/psxkoyfin/scrapper/DataWarehouse/omts2024-01-23.csv")
df1

,Date,SETTLEMENT DATE,BUYER,SELLER,SYMBOL CODE,COMPANY,TURNOVER,RATE,VALUES
0,23-Jan-24,24-Jan-24,525,145,HUBC,Hub Power Co.,500,119.14,59570


In [47]:
df2

,Date,SETTLEMENT DATE,MEMBER CODE,SYMBOL CODE,COMPANY,TURNOVER,RATE,VALUES
0,23-Jan-24,25-Jan-24,006,PIBTL,Pak Int.Bulk,150000,6.79,1018500
1,23-Jan-24,25-Jan-24,006,EFERT,Engro Fert.,3000,127.00,381000
2,23-Jan-24,25-Jan-24,006,BOP,B.O.Punjab,3500000,6.42,22470000
3,23-Jan-24,25-Jan-24,006,NCL,Nishat (Chun.),1450000,26.85,38932500
4,23-Jan-24,25-Jan-24,006,POWER,Power Cement,885000,5.43,4805550
5,23-Jan-24,25-Jan-24,006,THCCL,Thatta Cement,250000,18.30,4575000
6,23-Jan-24,25-Jan-24,050,WTL,WorldCall Telecom,5000000,1.31,6550000
7,23-Jan-24,23-Jan-24,084,ADAMS,Adam SugarXD,1000,34.00,34000
8,23-Jan-24,23-Jan-24,084,TGL,Tariq Glass Ind.,10,98.00,980
9,23-Jan-24,30-Jan-24,120,SML,Shakarganj Limited,425000,38.85,16511250


In [1]:
import execution as ex

In [16]:
df = ex.Execution().execution_by_extension()

  0%|          | 0/4 [00:00<?, ?it/s]
Extracting: 0it [00:00, ?it/s]
Extracting: 1it [00:00,  2.06it/s]
Extracting: 2it [00:00,  2.60it/s]
Extracting: 4it [00:01,  4.59it/s]
Extracting: 5it [00:02,  1.71it/s]
Extracting: 6it [00:04,  1.13it/s]
Extracting: 7it [00:05,  1.07s/it]
Extracting: 8it [00:09,  2.04s/it]
Extracting: 9it [00:11,  1.86s/it]
Extracting: 10it [00:12,  1.79s/it]
Extracting: 11it [00:14,  1.68s/it]
Extracting: 12it [00:16,  1.70s/it]
Extracting: 13it [00:17,  1.62s/it]
Extracting: 14it [00:18,  1.58s/it]
Extracting: 15it [00:20,  1.65s/it]
Extracting: 16it [00:21,  1.45s/it]
Extracting: 17it [00:22,  1.27s/it]
Extracting: 18it [00:23,  1.13s/it]
Extracting: 19it [00:24,  1.02s/it]
Extracting: 20it [00:24,  1.06it/s]
Extracting: 21it [00:25,  1.15it/s]
Extracting: 22it [00:26,  1.07it/s]
Extracting: 23it [00:27,  1.17it/s]
Extracting: 24it [00:28,  1.22it/s]
Extracting: 25it [00:28,  1.22it/s]
Extracting: 26it [00:29,  1.26it/s]
Extracting: 27it [00:30,  1.27it/s]
Ext

                 Main Board        Details                GEM Board  \
2   No. of Listed Companies            524  No. of Listed Companies   
3    Listed Paid up Capital      1669801 M   Listed Paid up Capital   
4     Market Capitalisation  9361781557196    Market Capitalisation   
5             Traded Volume      327591633            Traded Volume   
6              Traded Value    14308834932             Traded Value   
7                 Advancers            203                Advancers   
8                 Decliners            125                Decliners   
9                   Neutral             28                  Neutral   
10                    Total            356                    Total   

       Details  
2            3  
3       1832 M  
4   1877609370  
5            0  
6            0  
7            0  
8            0  
9            3  
10           3  
1                                    Company         Date   Time       Place  \
2     Pakistan National Shipping Corpor

In [19]:
import pandas as pd, logging
import pdfplumber


In [32]:
def extract_macro_view(pdf_file, targeted_pages_nums: list) -> pd.DataFrame:
    try:
        loop_ending_pattern = "PUBLICLY ISSUED DEBT SECURITIES"
        with open(pdf_file, 'r') as file:
            pdf_file  = pdfplumber.open(file)
        extracted_table = pdf_file.pages[targeted_pages_nums[0]].extract_table()
        print(extracted_table)
        all_rows = []

        for row in extracted_table:
            table = [cell for cell in row if cell is not None]
            if any(loop_ending_pattern in cell for cell in table):
                break
            all_rows.append(table)

        df = pd.DataFrame(all_rows)
        df.dropna(inplace=True) 
        df.columns = ["Main Board", "Details", "GEM Board", "Details"]
        df.replace(",", "", regex=True, inplace=True)
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")

In [67]:
from datetime import datetime

In [100]:
def extract_macro_view(pdf_file, targeted_pages_nums: list) -> pd.DataFrame:
    try:
        loop_ending_pattern = "PUBLICLY ISSUED DEBT SECURITIES"
        pdf_file = pdfplumber.open(pdf_file)
        metadaata = pdf_file.metadata
        metadaata = metadaata.get("CreationDate")[2:10]
        documents_creation_date = datetime.strptime(metadaata, "%Y%m%d").date()
        extracted_table = pdf_file.pages[targeted_pages_nums[0]-1].extract_table()
        
        all_rows = []

        for row in extracted_table:
            table = [cell for cell in row if cell is not None]
            if any(loop_ending_pattern in cell for cell in table):
                break
            all_rows.append(table)
        # print(all_rows)
        df = pd.DataFrame(all_rows)
        df.dropna(inplace=True)
        df.replace(",", "", regex=True, inplace=True)
        # print(df)
        df.columns = ["Market", "Main Board", "Details", "GEM Board"]
        df = df.T
        df.drop("Details", axis=0, inplace=True)
        df.columns = df.iloc[0]
        df.drop(df.index[0], inplace=True)
        df.insert(0, "Date", documents_creation_date)
        df
        
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")

In [101]:
df = extract_macro_view("/Volumes/PSXDatabase/psxkoyfin/DataWarehouse/quote_202407feb.pdf", [2])

In [102]:
df

Market,Date,No. of Listed Companies,Listed Paid up Capital,Market Capitalisation,Traded Volume,Traded Value,Advancers,Decliners,Neutral,Total
Main Board,2024-02-07,524,1669801 M,9361781557196,327591633,14308834932,203,125,28,356
GEM Board,2024-02-07,3,1832 M,1877609370,0,0,0,0,3,3


In [85]:
df = df.T
df.drop("Details", axis=0, inplace=True)

In [87]:
update_df.drop("Details", axis=0, inplace=True)
update_df

,2,3,4,5,6,7,8,9,10
Market,No. of Listed Companies,Listed Paid up Capital,Market Capitalisation,Traded Volume,Traded Value,Advancers,Decliners,Neutral,Total
Main Board,524,1669801 M,9361781557196,327591633,14308834932,203,125,28,356
GEM Board,3,1832 M,1877609370,0,0,0,0,3,3


In [5]:


def extract_macro_view(pdf_file_path, targeted_pages_nums: list) -> pd.DataFrame:
    try:
        loop_ending_pattern = "PUBLICLY ISSUED DEBT SECURITIES"
        
        with pdfplumber.open(pdf_file_path) as pdf_file:
            metadata = pdf_file.metadata
            creation_date_string = metadata.get("CreationDate")
            creation_date = datetime.datetime.strptime(creation_date_string[2:10], "%Y%m%d").date()

            extracted_table = pdf_file.pages[targeted_pages_nums[0] - 1].extract_table()
        
        all_rows = []

        for row in extracted_table:
            table = [cell for cell in row if cell is not None]
            if any(loop_ending_pattern in cell for cell in table):
                break
            all_rows.append(table)

        df = pd.DataFrame(all_rows)
        df.dropna(inplace=True)
        df.replace(",", "", regex=True, inplace=True)
        df.columns = ["Market", "Main Board", "Details", "GEM Board"]
        df = df.T
        df.drop("Details", axis=0, inplace=True)
        df.insert(0, "Date", creation_date)
        df.columns = df.iloc[0]
        df.drop(df.index[0], inplace=True)
        # df.reset_index(drop=True, inplace=True)
        
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")

# Example usage:
df = extract_macro_view("/Volumes/PSXDatabase/psxkoyfin/DataWarehouse/quote_202407feb.pdf", [2])


In [46]:
import logging
import datetime
import pdfplumber
import pandas as pd
import numpy as np

In [76]:
def extract_event_table(pdf_file, targeted_pages_nums: list) -> pd.DataFrame:
    """Extracts a table from the PDF based on the page numbers provided."""
    try:
        if len(targeted_pages_nums) == 1:
            target_table = pdfplumber.open(pdf_file).pages[
                targeted_pages_nums[0] - 1
            ].extract_table()  # -1 as page numbers start from 1 not 0
            tabledf = pd.DataFrame(target_table)
            tabledf.dropna(inplace=True)
            tabledf.columns = tabledf.iloc[0]
            tabledf.drop(tabledf.index[0], inplace=True)
            df = tabledf.iloc[:, 1:]
            df.insert(0, "FK", 'company_code')
            date_col = df['Date'].astype(str)
            time_col = df['Time'].astype(str)
            df['Datetime'] = date_col + ' ' + time_col
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df.drop(columns=['Date', 'Time'], inplace=True)
        else:
            df = pd.DataFrame()
            for page in range(targeted_pages_nums[0] - 1, targeted_pages_nums[1]):
                extracted_table_page = pdfplumber.open(pdf_file).pages[page]
                extracted_table = extracted_table_page.extract_table()
                extracted_table = pd.DataFrame(extracted_table)
                extracted_table.dropna(inplace=True)
                
                extracted_table.columns = extracted_table.iloc[0]
                extracted_table.drop(extracted_table.index[0], inplace=True)
                table = extracted_table.iloc[:, 1:]

                df = pd.concat([df, table], axis=0)
        return df
    except Exception as e:
        logging.error(f"Error Message: {e}")
        return pd.DataFrame()


In [77]:
df = extract_event_table("/Volumes/PSXDatabase/psxkoyfin/DataWarehouse/quote_202407feb.pdf", [30])

/var/folders/v5/clq57c8s10d3v7mfxdx4pd1w0000gn/T/ipykernel_96578/2644164307.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Datetime'] = date_col + ' ' + time_col
/var/folders/v5/clq57c8s10d3v7mfxdx4pd1w0000gn/T/ipykernel_96578/2644164307.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Datetime'] = pd.to_datetime(df['Datetime'])
/var/folders/v5/clq57c8s10d3v7mfxdx4pd1w0000gn/T/ipykernel_96578/2644164307.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [78]:
df

1,FK,Company,Place,Subject,Datetime
2,company_code,Pakistan National Shipping Corporation,Karachi,To consider the matter other than Financial Re...,2024-02-09 14:30:00
3,company_code,Crescent Cotton Mills Limited,Faisalabad,To consider the matter other than Financial Re...,2024-02-09 11:00:00
4,company_code,Al Meezan Investment Funds Limited,Karachi,Half Yearly Accounts for the Period ended Dece...,2024-02-09 14:30:00
5,company_code,Khalid Siraj Textile Mills,Lahore,Half Yearly Accounts for the Period ended Dece...,2024-02-09 09:30:00
6,company_code,Meezan Pakistan ETF,Karachi,Half Yearly Accounts for the Period ended Dece...,2024-02-09 14:30:00
7,company_code,Crescent Fibres Limited,Karachi,To consider the matter other than Financial Re...,2024-02-09 11:00:00
8,company_code,Chakwal Spinning Mills Limited,Lahore,To consider the matter other than Financial Re...,2024-02-10 11:00:00
9,company_code,Agritech Limited,Lahore,To consider the matter other than Financial Re...,2024-02-12 10:30:00
10,company_code,NetSol Technologies Limited,Lahore,Half Yearly Accounts for the Period ended Dece...,2024-02-12 14:00:00
11,company_code,Ali Asghar Textile Mills Limited,Karachi,Half Yearly Accounts for the Period ended Dece...,2024-02-12 14:00:00


In [ ]:
import pandas as pd

from finapp.psxkoyfin.models import (
                                    Country, City, CountryExchange, CountryExchangeSector,
                                    SectorStock, SectorStockinfo, SixMonthSummary, EventSummary,
                                    MacroView, IndexComposition, SectorSummary, OffMarketBroker,
                                    OffMarketClient, FutureOpenInterest, DailyStockData,
                                    )


def insert_data_from_dataframes(dataframes):
    for df_name, df in dataframes.items():
        if df_name == 'Country':
            for _, row in df.iterrows():
                country = Country.objects.create(
                    country_name=row['country_name'],
                    country_status=row['country_status']
                )
                print(f"Inserted {country} into Country model")
        elif df_name == 'City':
            for _, row in df.iterrows():
                country_id = row['country_id']  # Assuming you have country_id in the DataFrame
                country = Country.objects.get(pk=country_id)
                city = City.objects.create(
                    city_name=row['city_name'],
                    country=country,
                    added_datetime=row['added_datetime']
                )
                print(f"Inserted {city} into City model")
        elif df_name == 'CountryExchange':
            for _, row in df.iterrows():
                country_id = row['country_id']  # Assuming you have country_id in the DataFrame
                country = Country.objects.get(pk=country_id)
                country_exchange = CountryExchange.objects.create(
                    country_exchange_symbol=row['country_exchange_symbol'],
                    country_exchange_name=row['country_exchange_name'],
                    country_exchange_status=row['country_exchange_status'],
                    added_datetime=row['added_datetime'],
                    country=country
                )
                print(f"Inserted {country_exchange} into CountryExchange model")
        elif df_name == 'CountryExchangeSector':
            for _, row in df.iterrows():
                country_exchange_id = row['country_exchange_id']  # Assuming you have country_exchange_id in the DataFrame
                country_exchange = CountryExchange.objects.get(pk=country_exchange_id)
                country_exchange_sector = CountryExchangeSector.objects.create(
                    sector_code=row['sector_code'],
                    sector_name=row['sector_name'],
                    added_datetime=row['added_datetime'],
                    country_exchange=country_exchange
                )
                print(f"Inserted {country_exchange_sector} into CountryExchangeSector model")
        elif df_name == 'SectorStock':
            for _, row in df.iterrows():
                exchange_sector_id = row['exchange_sector_id']  # Assuming you have exchange_sector_id in the DataFrame
                exchange_sector = CountryExchangeSector.objects.get(pk=exchange_sector_id)
                sector_stock = SectorStock.objects.create(
                    stock_symbol=row['stock_symbol'],
                    stock_name=row['stock_name'],
                    stock_status=row['stock_status'],
                    added_datetime=row['added_datetime'],
                    exchange_sector=exchange_sector
                )
                print(f"Inserted {sector_stock} into SectorStock model")
        elif df_name == 'SectorStockinfo':
            for _, row in df.iterrows():
                stock_id = row['stock_id']  # Assuming you have stock_id in the DataFrame
                stock = SectorStock.objects.get(pk=stock_id)
                sector_stockinfo = SectorStockinfo.objects.create(
                    stock=stock,
                    face_value=row['face_value'],
                    lot_size=row['lot_size'],
                    registered_date=row['registered_date'],
                    year_end_date=row['year_end_date'],
                    number_of_shares=row['number_of_shares'],
                    free_float_shares=row['free_float_shares'],
                    percentage_free_float_shares=row['percentage_free_float_shares'],
                    number_of_shareholder=row['number_of_shareholder']
                )
                print(f"Inserted {sector_stockinfo} into SectorStockinfo model")
        elif df_name == 'MacroView':
            for _, row in df.iterrows():
                country_exchange_id = row['country_exchange_id']  # Assuming you have country_exchange_id in the DataFrame
                country_exchange = CountryExchange.objects.get(pk=country_exchange_id)
                macro_view = MacroView.objects.create(
                    market=row['market'],
                    date=row['date'],
                    num_listed_companies=row['num_listed_companies'],
                    listed_paid_up_capital=row['listed_paid_up_capital'],
                    market_capitalization=row['market_capitalization'],
                    traded_volume=row['traded_volume'],
                    advancers=row['advancers'],
                    decliners=row['decliners'],
                    neutral=row['neutral'],
                    total=row['total'],
                    country_exchange=country_exchange
                )
                print(f"Inserted {macro_view} into MacroView model")
        elif df_name == 'EventSummary':
            for _, row in df.iterrows():
                sector_stock_id = row['sector_stock_id']  # Assuming you have sector_stock_id in the DataFrame
                sector_stock = SectorStock.objects.get(pk=sector_stock_id)
                city_id = row['city_id']  # Assuming you have city_id in the DataFrame
                city = City.objects.get(pk=city_id)
                event_summary = EventSummary.objects.create(
                    sector_stock=sector_stock,
                    date_time=row['date_time'],
                    city=city,
                    subject=row['subject']
                )
                print(f"Inserted {event_summary} into EventSummary model")

# Assuming you have a dictionary of DataFrames where keys are the model names
# and values are the corresponding DataFrames
dataframes = {
    'Country': country_df,
    'City': city_df,
    'CountryExchange': country_exchange_df,
    'CountryExchangeSector': country_exchange_sector_df,
    'SectorStock': sector_stock_df,
    'SectorStockinfo': sector_stockinfo_df,
    'MacroView': macro_view_df,
    'EventSummary': event_summary_df
}

# Call the function to insert data into models
insert_data_from_dataframes(dataframes)
